In [1]:
# IMPORT 

import csv
from biomart import BiomartServer
import pandas as pd
import openpyxl

In [2]:
# DATASET

## biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['rnorvegicus_gene_ensembl']

In [3]:
# FUNCTIONS

def gene_dictionary(index,
                    gene_name,
                    gene_list_number,
                    gene_list_id,
                    source,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    organism,
                    alias,
                    direction_string):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Gene_list_number' : gene_list_number,
        'Gene_list_index' : gene_list_id,
        'Source' : source,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'Organism' : organism,
        'HGNC_symbol' : hgnc_symbol,
        'Alias' : alias,
        'Info' : direction_string
    }
    
    return dictionary


  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'external_gene_name':[mgi_symbol]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  


def biomartHumanOrthologs(mgi_symbol):
    
    attributes = ['hsapiens_homolog_associated_gene_name']
    filters = {'external_gene_name':[mgi_symbol]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
    
    return values  
    
def biomartParametersbyEnsembl(ensembl_id):

    attributes = ['external_gene_name',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'ensembl_gene_id':[ensembl_gene_id]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  

def biomartHumanOrthologsbyEnsembl(ensembl_id):

    attributes = ['hsapiens_homolog_associated_gene_name']

    filters = {'ensembl_gene_id':[ensembl_gene_id]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  




def alias_and_official(ls_notResponse,ls_row_10,ls_row_2):

    ls_response = []
    ls_response2 = []
    
    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                t_strings = temp.split("|")
                if word in t_strings:
                    ls_response.append([word, i, ls_row_1[i]])

    ls_response.insert(0, ['gene_name','alias_index','mgi_id'])         # print(len(ls_response))


    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                if word == temp:
                    ls_response2.append([word, i, ls_row_1[i]])
                    

    ls_response.insert(0, ['gene_name','official_index','mgi_id'])      # print(len(ls_response2))

    ls_response_3 = ls_response
    
    for response in ls_response2:
        if response not in ls_response_3:
            ls_response_3.append(response)
        else: print(response)

    ls_response_3.pop(0)
    ls_response_3.pop(0)
    ls_response.insert(0, ['gene_name','official/alias_index','mgi_id'])
    
    return ls_response_3

def biomartParameters_mgi(mgi_id):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']
    filters = {'mgi_id':[mgi_id]}                     # gene_name = mgi_id
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
    
    return values  


def biomartHumanOrthologs_mgi(mgi_id):
    attributes = ['hsapiens_homolog_associated_gene_name']
    
    filters = {'mgi_id':[mgi_id]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
    
    return values  


def updateCellswithAlias(mgi_file_path, dictionary_file_path, file_path):
    
    # Load MGI file and open workbook
    wb_mgi = openpyxl.load_workbook(mgi_file_path)
    ws_mgi = wb_mgi.active

    ls_row_alias = [ws_mgi.cell(row=i,column=10).value for i in range(2,ws_mgi.max_row+1)]
    ls_row_g_name = [ws_mgi.cell(row=i,column=2).value for i in range(2,ws_mgi.max_row+1)] 

    # Load dictionary file and open workbook
    workbook = openpyxl.load_workbook(dictionary_file_path)
    sheet = workbook.active

    ls_row_2 = [sheet.cell(row=i,column=2).value for i in range(2,sheet.max_row+1)]
    ls_row_9 = [sheet.cell(row=i,column=11).value for i in range(2,sheet.max_row+1)]

    
    length = len(ls_row_2) + 2

    # Modify the desired cell
    for i in range(length):
        K_cel = 'K' + str(i + 2)

        for k in range(len(ls_row_g_name)):
            try:
                if str(ls_row_g_name[k]).lower() == str(ls_row_2[i]).lower():
                    if ls_row_alias[k] is not None:
                        sheet[K_cel] = ls_row_alias[k]
                    else:
                        continue
            except:
                continue

    # Save the file
    workbook.save(file_path)
    
    
def updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path):
    
    wb_info = openpyxl.load_workbook(source_file_path)
    ws_info = wb_info[source_sheet]
    ls_row_g_name = [ws_info.cell(row=i, column=3).value for i in range(2, ws_info.max_row + 1)]


    wb_alias = openpyxl.load_workbook(alias_file_path)
    ws_alias = wb_alias.active
    ls_row_2 = [ws_alias.cell(row=i, column=2).value for i in range(2, ws_alias.max_row + 1)]


    matched_indices = set()               # to keep track of matched id

    for j, (g_name, row_2) in enumerate(zip(ls_row_g_name, ls_row_2), start=2):
        if j not in matched_indices and str(g_name).lower() == str(row_2).lower():
            print(str(g_name).lower(), str(row_2).lower())
            L_cel = 'L' + str(j)
            tempInfo = str(ws_alias[L_cel].value)
            matched_indices.add(j)        # add matched index to set - avoid duplicate

### the number and type of items in the INFO column depends on the source publication


            ws_alias[L_cel] = ''
            ws_alias[L_cel] = ('Regulation:' + str(tempInfo).upper() + '|'
                            + 'Log2Ratio:' + str(ws_info['E' + str(j)].value) + '|'
                            + 'tissue:' + str(ws_info['H' + str(j)].value) + '|'
                            + 'cell:' + str(ws_info['I' + str(j)].value) + '|'
                            + 'enviroroment:' + str(ws_info['J' + str(j)].value) + '|'
                            + 'treatment:' + str(ws_info['K' + str(j)].value) + '|'
                            + 'dose:' + str(ws_info['L' + str(j)].value) + '|'
                            + 'time:' + str(ws_info['M' + str(j)].value) + '|'
                            + 'FDR_threshold:' + str(ws_info['N' + str(j)].value) + '|'
                            + 'method:' + str(ws_info['O' + str(j)].value) + '|'
                            + 'FDR:' + str(ws_info['P' + str(j)].value))

    # Save the file
    wb_alias.save(info_file_path)


In [4]:
# LOAD START FILES

## PUBLICATION SOURCE
wb = openpyxl.load_workbook('../ifpan-GR-database-papers.xlsx')
ws = wb['nervous-cells']

publication_geneName = [ws.cell(row=i,column=3).value for i in range(2,ws.max_row+1)]
direction = [ws.cell(row=i,column=6).value for i in range(2,ws.max_row+1)]
Ensembl_id = [ws.cell(row=i,column=4).value for i in range(2,ws.max_row+1)]


## MGI
wb_1 = openpyxl.load_workbook(r'../MGI_EntrezGene.xlsx')
ws_1 = wb_1.active

ls_row_10 = [ws_1.cell(row=i,column=10).value for i in range(2,ws_1.max_row+1)]
ls_row_2 = [ws_1.cell(row=i,column=2).value for i in range(2,ws_1.max_row+1)]
ls_row_1 = [ws_1.cell(row=i,column=1).value for i in range(2,ws_1.max_row+1)]  

In [6]:
# LISTS

dictionary = {}
ls_geneDictionaries = []
ls_notResponse = []
ls_notResponse_after = []


In [7]:
# DICTIONARY

for i in range(len(publication_geneName)):
    gene_name = publication_geneName[i]
    direction_string = direction[i]   # => info
    
    # VARIABLES
    index = i + 1
    gene_list_number = 31
    gene_list_id =  'all_sig_genes_24147833'      # => cluster
    source = 'PMID: 24147833'
    organism = 'rat'
    print(direction_string)
    alias = ''
    
    if gene_name == '':
        continue
    
    
### "NA" GENE NAMES

    elif gene_name == 'NA':

        ensembl_gene_id = Ensembl_id[i]
        external_gene_name_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []

        ls_biomartParametersbyEnsembl = biomartParametersbyEnsembl(ensembl_gene_id)
        if not ls_biomartParametersbyEnsembl:
            print('no ensembl')


        for ls in ls_biomartParametersbyEnsembl:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            external_gene_name_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        gene_name = '|'.join(list(set(filter(None, external_gene_name_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))

        print(gene_name)

        # ORTHOLOGS
        ls_biomartHumanOrthologsbyEnsembl = biomartHumanOrthologsbyEnsembl(ensembl_gene_id)
        if not ls_biomartHumanOrthologsbyEnsembl:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologsbyEnsembl[0][0]
        
        temp_gene_dictionary = gene_dictionary(index,
                                       gene_name,
                                       gene_list_number,
                                       gene_list_id,
                                       source,
                                       ensembl_gene_id,
                                       ensembl_transcript_id, 
                                       refseq_mrna,
                                       hgnc_symbol,
                                       organism,
                                       alias,
                                       direction_string)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)    

### GENE NAMES

    else:

        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []


        ls_biomartParameters = biomartParameters(gene_name)
        if not ls_biomartParameters:
            ls_notResponse.append(gene_name)
            print(ls_notResponse)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))


        # ORTHOLOGS
        ls_biomartHumanOrthologs = biomartHumanOrthologs(gene_name)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        
        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               gene_list_number,
                                               gene_list_id,
                                               source,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               organism,
                                               alias,
                                               direction_string)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)





up
{'Index': 1, 'Gene_name': 'HTRA1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000020533', 'Ensembl_transcript_id': 'ENSRNOT00000111703|ENSRNOT00000027860|ENSRNOT00000115809', 'RefSeq_mRNA_id': 'NM_031721', 'Organism': 'rat', 'HGNC_symbol': 'HTRA1', 'Alias': '', 'Info': 'up'}
up
{'Index': 2, 'Gene_name': 'HOPX', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000024689', 'Ensembl_transcript_id': 'ENSRNOT00000049774', 'RefSeq_mRNA_id': 'NM_133621', 'Organism': 'rat', 'HGNC_symbol': 'HOPX', 'Alias': '', 'Info': 'up'}
up
{'Index': 3, 'Gene_name': 'ALDOC', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000011452', 'Ensembl_transcript_id': 'ENSRNOT00000015535|ENSRNOT00000046388|ENSRNOT00000086036', 'RefSeq_mRNA_id': 'NM_012497', 'Organism': 'rat', 'HGNC

{'Index': 25, 'Gene_name': 'CYP7B1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000009730', 'Ensembl_transcript_id': 'ENSRNOT00000013116|ENSRNOT00000082084', 'RefSeq_mRNA_id': 'NM_019138', 'Organism': 'rat', 'HGNC_symbol': 'CYP7B1', 'Alias': '', 'Info': 'down'}
down
{'Index': 26, 'Gene_name': 'TSPAN12', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000059016', 'Ensembl_transcript_id': 'ENSRNOT00000083938|ENSRNOT00000100250', 'RefSeq_mRNA_id': 'NM_001015026', 'Organism': 'rat', 'HGNC_symbol': 'TSPAN12', 'Alias': '', 'Info': 'down'}
up
{'Index': 27, 'Gene_name': 'CSAD', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000011573', 'Ensembl_transcript_id': 'ENSRNOT00000016205|ENSRNOT00000113521|ENSRNOT00000102299|ENSRNOT00000079678|ENSRNOT00000117956|ENS

['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED']
{'Index': 49, 'Gene_name': 'ARHGAP21_PREDICTED', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
down
{'Index': 50, 'Gene_name': 'APOE', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000018454', 'Ensembl_transcript_id': 'ENSRNOT00000108411|ENSRNOT00000080453|ENSRNOT00000091574|ENSRNOT00000099048|ENSRNOT00000041891', 'RefSeq_mRNA_id': 'NM_001270684|NM_138828|NM_001270682|NM_001270681|NM_001270683', 'Organism': 'rat', 'HGNC_symbol': 'APOE', 'Alias': '', 'Info': 'down'}
up
{'Index': 51, 'Gene_name': 'GPD1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'S

['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED']
{'Index': 70, 'Gene_name': 'PLTP_PREDICTED', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'down'}
down
{'Index': 71, 'Gene_name': 'GMFG', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000019838', 'Ensembl_transcript_id': 'ENSRNOT00000118600|ENSRNOT00000026891|ENSRNOT00000118560', 'RefSeq_mRNA_id': 'NM_181091', 'Organism': 'rat', 'HGNC_symbol': 'GMFG', 'Alias': '', 'Info': 'down'}
down
{'Index': 72, 'Gene_name': 'UCP2', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Sour

{'Index': 90, 'Gene_name': 'LYL1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000002950', 'Ensembl_transcript_id': 'ENSRNOT00000003980', 'RefSeq_mRNA_id': 'NM_001007677', 'Organism': 'rat', 'HGNC_symbol': 'LYL1', 'Alias': '', 'Info': 'down'}
down
['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED', 'LOC310926', 'ADFP', 'HLA-DMB', 'PEPD', 'LY86_PREDICTED', 'RT1-A1']
{'Index': 91, 'Gene_name': 'RT1-A1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'down'}
down
{'Index': 92, 'Gene_name': 'PSAP', 'Gene_list_number': 31, 'Gene_list_index': 'all_s

...

{'Index': 283, 'Gene_name': 'LOC361885', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
{'Index': 284, 'Gene_name': 'ATP1A2', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000007290', 'Ensembl_transcript_id': 'ENSRNOT00000118040|ENSRNOT00000058143|ENSRNOT00000108846', 'RefSeq_mRNA_id': 'NM_012505', 'Organism': 'rat', 'HGNC_symbol': 'ATP1A2', 'Alias': '', 'Info': 'up'}
up
['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED', 'LOC310926', 'ADFP', 'HLA-DMB', 'PEPD', 'LY86_PREDICTED', 'RT1-A1', 'GNS_PREDICTED', 'RGD1303130', 'FBXW4_PREDICTED'

{'Index': 289, 'Gene_name': 'LOC499564', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED', 'LOC310926', 'ADFP', 'HLA-DMB', 'PEPD', 'LY86_PREDICTED', 'RT1-A1', 'GNS_PREDICTED', 'RGD1303130', 'FBXW4_PREDICTED', 'RT1-A3', 'LOC310946', 'RGD1311381_PREDICTED', 'PRKCB1', 'FUCA', 'IFI27L', 'HARPB64', 'TMEPAI_PREDICTED', 'FHL3_PREDICTED', 'C2', 'BHLHB3', 'LOC497767', 'LOC306324', 'CREG_PREDICTED', 'AP1S2_PREDICTED', 'IFNGR2_PREDICTED', 'SH3BGRL3_PREDICTED', 'ARPC4_PREDICTED', 'TCFE3_PREDICTED', 'LOC500040', 'RGD1563276_PREDICTED', 'ISG12(B)', 'LOC499775', 'SART2_PREDICTED', 

{'Index': 295, 'Gene_name': 'LOC500586', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED', 'LOC310926', 'ADFP', 'HLA-DMB', 'PEPD', 'LY86_PREDICTED', 'RT1-A1', 'GNS_PREDICTED', 'RGD1303130', 'FBXW4_PREDICTED', 'RT1-A3', 'LOC310946', 'RGD1311381_PREDICTED', 'PRKCB1', 'FUCA', 'IFI27L', 'HARPB64', 'TMEPAI_PREDICTED', 'FHL3_PREDICTED', 'C2', 'BHLHB3', 'LOC497767', 'LOC306324', 'CREG_PREDICTED', 'AP1S2_PREDICTED', 'IFNGR2_PREDICTED', 'SH3BGRL3_PREDICTED', 'ARPC4_PREDICTED', 'TCFE3_PREDICTED', 'LOC500040', 'RGD1563276_PREDICTED', 'ISG12(B)', 'LOC499775', 'SART2_PREDICTED', 

{'Index': 300, 'Gene_name': 'LOC498245', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'rat', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
{'Index': 301, 'Gene_name': 'HTRA1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000020533', 'Ensembl_transcript_id': 'ENSRNOT00000111703|ENSRNOT00000027860|ENSRNOT00000115809', 'RefSeq_mRNA_id': 'NM_031721', 'Organism': 'rat', 'HGNC_symbol': 'HTRA1', 'Alias': '', 'Info': 'up'}
up
['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED', 'LOC310926', 'ADFP', 'HLA-DMB', 'PEPD', 'LY86_PREDICTED', 'RT1-A1', 'GNS_PREDICTED', 'RGD1303130', 'FBXW4_PREDICTED', 

{'Index': 309, 'Gene_name': 'GPD1', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000056457', 'Ensembl_transcript_id': 'ENSRNOT00000087662|ENSRNOT00000097280', 'RefSeq_mRNA_id': 'NM_022215', 'Organism': 'rat', 'HGNC_symbol': 'GPD1', 'Alias': '', 'Info': 'up'}
up
{'Index': 310, 'Gene_name': 'ALDOC', 'Gene_list_number': 31, 'Gene_list_index': 'all_sig_genes_24147833', 'Source': 'PMID: 24147833', 'Ensembl_gene_id': 'ENSRNOG00000011452', 'Ensembl_transcript_id': 'ENSRNOT00000015535|ENSRNOT00000046388|ENSRNOT00000086036', 'RefSeq_mRNA_id': 'NM_012497', 'Organism': 'rat', 'HGNC_symbol': 'ALDOC', 'Alias': '', 'Info': 'up'}


In [8]:
print(ls_notResponse)

['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED', 'LOC310926', 'ADFP', 'HLA-DMB', 'PEPD', 'LY86_PREDICTED', 'RT1-A1', 'GNS_PREDICTED', 'RGD1303130', 'FBXW4_PREDICTED', 'RT1-A3', 'LOC310946', 'RGD1311381_PREDICTED', 'PRKCB1', 'FUCA', 'IFI27L', 'HARPB64', 'TMEPAI_PREDICTED', 'FHL3_PREDICTED', 'C2', 'BHLHB3', 'LOC497767', 'LOC306324', 'CREG_PREDICTED', 'AP1S2_PREDICTED', 'IFNGR2_PREDICTED', 'SH3BGRL3_PREDICTED', 'ARPC4_PREDICTED', 'TCFE3_PREDICTED', 'LOC500040', 'RGD1563276_PREDICTED', 'ISG12(B)', 'LOC499775', 'SART2_PREDICTED', 'RGD1560373_PREDICTED', 'TNIP1_PREDICTED', 'RRAGC_PREDICTED', 'MGC72614', 'ARPC3_PREDICTED', 'TMEM55B', 'SNX2_PREDICTED', 'DNASE2A', 'GLRX1', 'MGC94190', 'CERK_PREDICTED', 'RGD1564762_PREDICTED', 'CTTNBP2NL_PREDICTED', 'LOC309557', 'RGD1310868_PREDICTED', 'SWAP70_PREDICTED', 'LOC

In [9]:
# SCORES 

### from DICTIONARY to file
data = ls_geneDictionaries
df = pd.DataFrame.from_dict(data)
df.to_excel(r'.\Dictionary.xlsx', index=False)

### from ls_notResponse to file
data1 = ls_notResponse
df1 = pd.DataFrame.from_dict(data1)
df1.to_excel(r'.\notResponse.xlsx', index=False)

# LOAD 
# notRESPONSE
wb_2 = openpyxl.load_workbook(r'.\notResponse.xlsx')
ws_2 = wb_2.active
ls_notResponse_v2 = [ws_2.cell(row=i,column=1).value for i in range(2,ws_2.max_row+1)]


### from alias_and_official to file
data2 = alias_and_official(ls_notResponse, ls_row_10, ls_row_2)
df2 = pd.DataFrame.from_dict(data2)
df2.to_excel(r'.\responsewithMGI.xlsx', index=False)


# LOAD responsewithMGI
wb_3 = openpyxl.load_workbook(r'.\responsewithMGI.xlsx')
ws_3 = wb_3.active
mgi_id = [ws_3.cell(row=i,column=3).value for i in range(3,ws_3.max_row+1)]
gene_name_v2 = [ws_3.cell(row=i,column=1).value for i in range(3,ws_3.max_row+1)]

['sgk', 36747, 'MGI:1340062']
['c2', 608914, 'MGI:7119820']
['tmem55b', 80264, 'MGI:2448501']


In [10]:
print (ls_notResponse)

['MT1A', 'RGD1566244_PREDICTED', 'FBLN1_PREDICTED', 'CTGF', 'EIF1B_PREDICTED', 'SGK', 'LOC311772', 'NUPR1', 'RHOB', 'LOC310926', 'ARHGAP21_PREDICTED', 'MSR2_PREDICTED', 'PFC_MAPPED', 'TREM2_PREDICTED', 'CD83_PREDICTED', 'PLTP_PREDICTED', 'LOC310926', 'ADFP', 'HLA-DMB', 'PEPD', 'LY86_PREDICTED', 'RT1-A1', 'GNS_PREDICTED', 'RGD1303130', 'FBXW4_PREDICTED', 'RT1-A3', 'LOC310946', 'RGD1311381_PREDICTED', 'PRKCB1', 'FUCA', 'IFI27L', 'HARPB64', 'TMEPAI_PREDICTED', 'FHL3_PREDICTED', 'C2', 'BHLHB3', 'LOC497767', 'LOC306324', 'CREG_PREDICTED', 'AP1S2_PREDICTED', 'IFNGR2_PREDICTED', 'SH3BGRL3_PREDICTED', 'ARPC4_PREDICTED', 'TCFE3_PREDICTED', 'LOC500040', 'RGD1563276_PREDICTED', 'ISG12(B)', 'LOC499775', 'SART2_PREDICTED', 'RGD1560373_PREDICTED', 'TNIP1_PREDICTED', 'RRAGC_PREDICTED', 'MGC72614', 'ARPC3_PREDICTED', 'TMEM55B', 'SNX2_PREDICTED', 'DNASE2A', 'GLRX1', 'MGC94190', 'CERK_PREDICTED', 'RGD1564762_PREDICTED', 'CTTNBP2NL_PREDICTED', 'LOC309557', 'RGD1310868_PREDICTED', 'SWAP70_PREDICTED', 'LOC

In [11]:
# SECOND DICTIONARY

for i in range(len(mgi_id)):
    mgi_ID = mgi_id[i]
    gene_name = gene_name_v2[i]
    print(mgi_ID)
    
    if gene_name != '':

        # variables
        alias = ''

        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []
        

        ls_biomartParameters_mgi = biomartParameters_mgi(mgi_ID)
        
        print(ls_biomartParameters)
        
        if not ls_biomartParameters:
            ls_notResponse_after.append(mgi_ID)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
         
        
        ls_biomartHumanOrthologs = biomartHumanOrthologs_mgi(mgi_ID)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]       
        
        
        for k in range(len(ls_geneDictionaries)):
            temp = str(ls_geneDictionaries[k]['Gene_name']).lower()

            if temp == gene_name:
                print('response')
                ls_geneDictionaries[k]['Ensembl_gene_id'] = ensembl_gene_id
                ls_geneDictionaries[k]['Ensembl_transcript_id'] = ensembl_transcript_id
                ls_geneDictionaries[k]['RefSeq_mRNA_id'] = refseq_mrna
                ls_geneDictionaries[k]['HGNC_symbol'] = hgnc_symbol
                


MGI:102725
[['ENSRNOG00000011452', 'ENSRNOT00000015535', 'NM_012497'], ['ENSRNOG00000011452', 'ENSRNOT00000046388', ''], ['ENSRNOG00000011452', 'ENSRNOT00000086036', '']]
response
response
MGI:95537
[['ENSRNOG00000011452', 'ENSRNOT00000015535', 'NM_012497'], ['ENSRNOG00000011452', 'ENSRNOT00000046388', ''], ['ENSRNOG00000011452', 'ENSRNOT00000086036', '']]
response
response
MGI:1340062
[['ENSRNOG00000011452', 'ENSRNOT00000015535', 'NM_012497'], ['ENSRNOG00000011452', 'ENSRNOT00000046388', ''], ['ENSRNOG00000011452', 'ENSRNOT00000086036', '']]
response
MGI:87920
[['ENSRNOG00000011452', 'ENSRNOT00000015535', 'NM_012497'], ['ENSRNOG00000011452', 'ENSRNOT00000046388', ''], ['ENSRNOG00000011452', 'ENSRNOT00000086036', '']]
response
MGI:97596
[['ENSRNOG00000011452', 'ENSRNOT00000015535', 'NM_012497'], ['ENSRNOG00000011452', 'ENSRNOT00000046388', ''], ['ENSRNOG00000011452', 'ENSRNOT00000086036', '']]
response
MGI:1347005
[['ENSRNOG00000011452', 'ENSRNOT00000015535', 'NM_012497'], ['ENSRNOG000

In [12]:
# SCORES 

### from SECOND DICTIONARY to file

data3 = ls_geneDictionaries
df3 = pd.DataFrame.from_dict(data3)
df3.to_excel(r'.\secondDictionary.xlsx', index=False)
df3.to_csv(r'.\secondDictionary.tsv', sep="\t")

data4 = ls_notResponse_after
df4 = pd.DataFrame.from_dict(data4)
df4.to_excel(r'.\ls_notResponse_after.xlsx', index=False)



In [13]:
# Add ALIAS

mgi_file_path = r'..\MGI_EntrezGene.xlsx'
dictionary_file_path = r'.\secondDictionary.xlsx'
save_file_path = r'.\withAlias.xlsx'


updateCellswithAlias(mgi_file_path, dictionary_file_path, save_file_path)


In [14]:
# Add INFO

source_file_path = r'..\ifpan-GR-database-papers.xlsx'
source_sheet = 'nervous-cells'
alias_file_path = '.\withAlias.xlsx'
info_file_path = r'.\withINFO.xlsx'

updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path)


htra1 htra1
hopx hopx
aldoc aldoc
mt1a mt1a
rgd1566244_predicted rgd1566244_predicted
sult1a1 sult1a1
phyhd1 phyhd1
cdo1 cdo1
pawr pawr
ednrb ednrb
fkbp5 fkbp5
fbln1_predicted fbln1_predicted
ctgf ctgf
htra1 htra1
ctsc ctsc
tsc22d3 tsc22d3
eif1b_predicted eif1b_predicted
sgk sgk
loc311772 loc311772
stmn2 stmn2
grina grina
plekhb1 plekhb1
cryab cryab
ddit4 ddit4
cyp7b1 cyp7b1
tspan12 tspan12
csad csad
resp18 resp18
gda gda
prdx6 prdx6
aurkb aurkb
klf9 klf9
nupr1 nupr1
hopx hopx
oaf oaf
spp1 spp1
pnoc pnoc
oplah oplah
cxcl1 cxcl1
rhob rhob
atp1a2 atp1a2
loc310926 loc310926
slc7a3 slc7a3
htra1 htra1
cst3 cst3
plcl1 plcl1
gap43 gap43
cryab cryab
arhgap21_predicted arhgap21_predicted
apoe apoe
gpd1 gpd1
rcan2 rcan2
thtpa thtpa
fcgr3a fcgr3a
msr2_predicted msr2_predicted
coro1a coro1a
pfc_mapped pfc_mapped
rac2 rac2
lpl lpl
apobec1 apobec1
laptm5 laptm5
trem2_predicted trem2_predicted
ctsc ctsc
ifi30 ifi30
cd83_predicted cd83_predicted
hprt1 hprt1
ehd4 ehd4
lcp1 lcp1
tmem176b tmem176b
pltp_p

In [ ]:
                                ''' + '|'
                                 + '|'
                                +'TTEST.NEURO.ALD:' + str(ws_mgi[S_cel].value) + '|'
                                +'TTEST.NEURO.DHT:' + str(ws_mgi[T_cel].value) + '|'
                                +'TTEST.NEURO.PRG:' + str(ws_mgi[U_cel].value) + '|'
                                +'TTEST.NEURO.ESR:' + str(ws_mgi[V_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DEX:' + str(ws_mgi[W_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ALD:' + str(ws_mgi[X_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DHT:' + str(ws_mgi[Y_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.PRG:' + str(ws_mgi[Z_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ESR:' + str(ws_mgi[AA_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DEX:' + str(ws_mgi[AB_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ALD:' + str(ws_mgi[AC_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DHT:' + str(ws_mgi[AD_cel].value) + '|'
                                +'FDR_TTEST.NEURO.PRG:' + str(ws_mgi[AE_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ESR:' + str(ws_mgi[AF_cel].value) + '|'
                                +'FOLD.ASTRO.DEX:' + str(ws_mgi[AG_cel].value) + '|'
                                +'FOLD.ASTRO.ALD:' + str(ws_mgi[AH_cel].value) + '|'
                                +'FOLD.ASTRO.DHT:' + str(ws_mgi[AI_cel].value) + '|'
                                +'FOLD.ASTRO.PRG:' + str(ws_mgi[AJ_cel].value) + '|'
                                +'FOLD.ASTRO.ESR:' + str(ws_mgi[AK_cel].value) + '|'
                                +'FOLD.NEURO.DEX:' + str(ws_mgi[AL_cel].value) + '|'
                                +'FOLD.NEURO.ALD:' + str(ws_mgi[AM_cel].value) + '|'
                                +'FOLD.NEURO.DHT:' + str(ws_mgi[AN_cel].value) + '|'
                                +'FOLD.NEURO.PRG:' + str(ws_mgi[AO_cel].value) + '|'
                                +'FOLD.NEURO.ESR:' + str(ws_mgi[AP_cel].value))'''